In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
import warnings
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
import time
import os
warnings.filterwarnings('ignore')
import wavio
import librosa
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,LSTM,TimeDistributed,Bidirectional
from keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from keras.callbacks import LearningRateScheduler
import sklearn


In [2]:
filename_list = ['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']


In [3]:
features,labels = np.empty((0,40,44)),np.empty(0)
for i in range(30):
    filePath = "D:\\下载\\train\\%s" % filename_list[i]
    fl = os.listdir(filePath)
    print(len(fl))
    for j in range(len(fl)):
        wavpath = filePath + '\\' + fl[j]
        sig,sr = librosa.load(wavpath)
        #不够长度的信号进行补0
        x = np.pad(sig,(0,22050-sig.shape[0]),'constant')
        #print(x.shape)
        #提取信号的mfcc并进行标准化
        a = librosa.feature.mfcc(x,sr,n_mfcc=40)
        norm_mfccs = sklearn.preprocessing.scale(a,axis=1)
        features = np.append(features,norm_mfccs[None],axis=0)
        labels = np.append(labels,int(i))
        if j < 2:
            print(features.shape)
    print('*****%s*****'%i)



1537
(1, 40, 44)
(2, 40, 44)
*****0*****
1573
(1538, 40, 44)
(1539, 40, 44)
*****1*****
1567
(3111, 40, 44)
(3112, 40, 44)
*****2*****
1566
(4678, 40, 44)
(4679, 40, 44)
*****3*****
2106
(6244, 40, 44)
(6245, 40, 44)
*****4*****
2095
(8350, 40, 44)
(8351, 40, 44)
*****5*****
2086
(10445, 40, 44)
(10446, 40, 44)
*****6*****
2119
(12531, 40, 44)
(12532, 40, 44)
*****7*****
2121
(14650, 40, 44)
(14651, 40, 44)
*****8*****
1562
(16771, 40, 44)
(16772, 40, 44)
*****9*****
1600
(18333, 40, 44)
(18334, 40, 44)
*****10*****
2086
(19933, 40, 44)
(19934, 40, 44)
*****11*****
1584
(22019, 40, 44)
(22020, 40, 44)
*****12*****
2105
(23603, 40, 44)
(23604, 40, 44)
*****13*****
2123
(25708, 40, 44)
(25709, 40, 44)
*****14*****
2095
(27831, 40, 44)
(27832, 40, 44)
*****15*****
2121
(29926, 40, 44)
(29927, 40, 44)
*****16*****
2122
(32047, 40, 44)
(32048, 40, 44)
*****17*****
2108
(34169, 40, 44)
(34170, 40, 44)
*****18*****
2138
(36277, 40, 44)
(36278, 40, 44)
*****19*****
1548
(38415, 40, 44)
(38416,

In [4]:
features = np.empty((0,28,44))
for i in range(30):
    filePath = "D:\\下载\\train\\%s" % filename_list[i]
    fl = os.listdir(filePath)
    print(len(fl))
    for j in range(len(fl)):
        wavpath = filePath + '\\' + fl[j]
        x,sr = librosa.load(wavpath)
        #不够长度的信号进行补0
        sig = np.pad(x,(0,22050-x.shape[0]),'constant')
        a = librosa.feature.zero_crossing_rate(sig,sr)
        b = librosa.feature.spectral_centroid(sig,sr=sr)[0]
        a = np.vstack((a,b))
        b = librosa.feature.chroma_stft(sig,sr)
        a = np.vstack((a,b))
        b = librosa.feature.spectral_contrast(sig,sr)
        a = np.vstack((a,b))
        b = librosa.feature.spectral_bandwidth(sig,sr)
        a = np.vstack((a,b))
        b = librosa.feature.tonnetz(sig,sr)
        a = np.vstack((a,b))
        norm_a = sklearn.preprocessing.scale(a,axis=1)
        #print(norm_mfccs.shape)
        features = np.append(features,norm_a[None],axis=0)
        if j < 2:
            print(features.shape)
    print('*****%s*****'%i)


1537
(1, 28, 44)
(2, 28, 44)
*****0*****
1573
(1538, 28, 44)
(1539, 28, 44)
*****1*****
1567
(3111, 28, 44)
(3112, 28, 44)
*****2*****
1566
(4678, 28, 44)
(4679, 28, 44)
*****3*****
2106
(6244, 28, 44)
(6245, 28, 44)
*****4*****
2095
(8350, 28, 44)
(8351, 28, 44)
*****5*****
2086
(10445, 28, 44)
(10446, 28, 44)
*****6*****
2119
(12531, 28, 44)
(12532, 28, 44)
*****7*****
2121
(14650, 28, 44)
(14651, 28, 44)
*****8*****
1562
(16771, 28, 44)
(16772, 28, 44)
*****9*****
1600
(18333, 28, 44)
(18334, 28, 44)
*****10*****
2086
(19933, 28, 44)
(19934, 28, 44)
*****11*****
1584
(22019, 28, 44)
(22020, 28, 44)
*****12*****
2105
(23603, 28, 44)
(23604, 28, 44)
*****13*****
2123
(25708, 28, 44)
(25709, 28, 44)
*****14*****
2095
(27831, 28, 44)
(27832, 28, 44)
*****15*****
2121
(29926, 28, 44)
(29927, 28, 44)
*****16*****
2122
(32047, 28, 44)
(32048, 28, 44)
*****17*****
2108
(34169, 28, 44)
(34170, 28, 44)
*****18*****
2138
(36277, 28, 44)
(36278, 28, 44)
*****19*****
1548
(38415, 28, 44)
(38416,

In [5]:
#测试集
X_test = np.empty((0,40,44))
filePath = "D:\\下载\\test"
fl = os.listdir(filePath)
print(len(fl))
for j in range(len(fl)):
    wavpath = filePath + '\\' + fl[j]
    sig,sr = librosa.load(wavpath)
    #不够长度的信号进行补0
    x = np.pad(sig,(0,22050-sig.shape[0]),'constant')
    #print(x.shape)
    #提取信号的mfc并进行标准化
    mfcc = librosa.feature.mfcc(x,sr,n_mfcc=40)
    norm_mfccs = sklearn.preprocessing.scale(mfcc,axis=1)
    X_test = np.append(X_test,norm_mfccs[None],axis=0)
    if j < 2:
        print(X_test.shape)


6835
(1, 40, 44)
(2, 40, 44)


In [6]:
X_test = np.empty((0,28,44))
filePath = "D:\\下载\\test"
fl = os.listdir(filePath)
print(len(fl))
for j in range(len(fl)):
    wavpath = filePath + '\\' + fl[j]
    x,sr = librosa.load(wavpath)
    #不够长度的信号进行补0
    sig = np.pad(x,(0,22050-x.shape[0]),'constant')
    a = librosa.feature.zero_crossing_rate(sig,sr)
    b = librosa.feature.spectral_centroid(sig,sr=sr)[0]
    a = np.vstack((a,b))
    b = librosa.feature.chroma_stft(sig,sr)
    a = np.vstack((a,b))
    b = librosa.feature.spectral_contrast(sig,sr)
    a = np.vstack((a,b))
    b = librosa.feature.spectral_bandwidth(sig,sr)
    a = np.vstack((a,b))
    b = librosa.feature.tonnetz(sig,sr)
    a = np.vstack((a,b))
    norm_a = sklearn.preprocessing.scale(a,axis=1)
    #print(norm_mfccs.shape)
    X_test = np.append(X_test,norm_a[None],axis=0)
    if j < 2:
        print(X_test.shape)


6835
(1, 28, 44)
(2, 28, 44)


In [7]:
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)
print(X_test.shape)
nfold = 5
kf = KFold(n_splits=nfold, shuffle=True, random_state=2020)
prediction1 = np.zeros((len(X_test),30 ))
i = 0
for train_index, valid_index in kf.split(features, labels):
    print("\nFold {}".format(i + 1))
    train_x, train_y = features[train_index],labels[train_index]
    val_x, val_y = features[valid_index],labels[valid_index]
    train_x = train_x.reshape(train_x.shape[0],train_x.shape[1],train_x.shape[2],1)
    val_x = val_x.reshape(val_x.shape[0],val_x.shape[1],val_x.shape[2],1)
    print(train_x.shape)
    print(val_x.shape)
    train_y = to_categorical(train_y,30)
    val_y = to_categorical(val_y,30)
    print(train_y.shape)
    print(val_y.shape)
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), activation='relu',input_shape = train_x.shape[1:]))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25)) 
    model.add(Convolution2D(32, (3, 3),  activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(30, activation='softmax'))
    model.compile(optimizer='Adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
    model.summary(line_length=80)
    history = model.fit(train_x, train_y, epochs=35, batch_size=32, validation_data=(val_x, val_y))
    y1 = model.predict(X_test)
    prediction1 += ((y1)) / nfold
    i += 1


(6835, 28, 44, 1)

Fold 1
(46308, 28, 44, 1)
(11578, 28, 44, 1)
(46308, 30)
(11578, 30)
Model: "sequential"
________________________________________________________________________________
 Layer (type)                       Output Shape                    Param #     
 conv2d (Conv2D)                    (None, 26, 42, 32)              320         
                                                                                
 max_pooling2d (MaxPooling2D)       (None, 13, 21, 32)              0           
                                                                                
 dropout (Dropout)                  (None, 13, 21, 32)              0           
                                                                                
 conv2d_1 (Conv2D)                  (None, 11, 19, 32)              9248        
                                                                                
 max_pooling2d_1 (MaxPooling2D)     (None, 5, 9, 32)                0           
 

1448/1448 [==============================] - 36s 25ms/step - loss: 3.0154 - accuracy: 0.1494 - val_loss: 2.5486 - val_accuracy: 0.2867
Epoch 2/35
1448/1448 [==============================] - 35s 24ms/step - loss: 2.4506 - accuracy: 0.2997 - val_loss: 2.1613 - val_accuracy: 0.3859
Epoch 3/35
1448/1448 [==============================] - 34s 24ms/step - loss: 2.2067 - accuracy: 0.3621 - val_loss: 2.0018 - val_accuracy: 0.4387
Epoch 4/35
1448/1448 [==============================] - 34s 24ms/step - loss: 2.0904 - accuracy: 0.3933 - val_loss: 1.8881 - val_accuracy: 0.4644
Epoch 5/35
1448/1448 [==============================] - 36s 25ms/step - loss: 2.0083 - accuracy: 0.4170 - val_loss: 1.8240 - val_accuracy: 0.4843
Epoch 6/35
1448/1448 [==============================] - 36s 25ms/step - loss: 1.9445 - accuracy: 0.4319 - val_loss: 1.7781 - val_accuracy: 0.5000
Epoch 7/35
1448/1448 [==============================] - 35s 24ms/step - loss: 1.8863 - accuracy: 0.4473 - val_loss: 1.7194 - val_accura

1448/1448 [==============================] - 33s 23ms/step - loss: 1.6752 - accuracy: 0.5034 - val_loss: 1.5563 - val_accuracy: 0.5503
Epoch 18/35
1448/1448 [==============================] - 35s 24ms/step - loss: 1.6622 - accuracy: 0.5056 - val_loss: 1.5588 - val_accuracy: 0.5492
Epoch 19/35
1448/1448 [==============================] - 34s 24ms/step - loss: 1.6522 - accuracy: 0.5096 - val_loss: 1.5389 - val_accuracy: 0.5575
Epoch 20/35
1448/1448 [==============================] - 34s 24ms/step - loss: 1.6436 - accuracy: 0.5106 - val_loss: 1.5267 - val_accuracy: 0.5596
Epoch 21/35
1448/1448 [==============================] - 35s 24ms/step - loss: 1.6276 - accuracy: 0.5155 - val_loss: 1.5142 - val_accuracy: 0.5671
Epoch 22/35
1448/1448 [==============================] - 35s 24ms/step - loss: 1.6140 - accuracy: 0.5177 - val_loss: 1.5090 - val_accuracy: 0.5625
Epoch 23/35
1448/1448 [==============================] - 37s 25ms/step - loss: 1.6082 - accuracy: 0.5202 - val_loss: 1.5124 - val_

In [12]:
y_pred=[list(x).index(max(x)) for x in prediction1]
sub = pd.read_csv('D:\\下载\\submission.csv')
cnt = 0
result = [0 for i in range(6835)]
for i in range(6835):
    ss = sub.iloc[i]['file_name']
    for j in range(6835):
        if fl[j] == ss:
            result[i] = y_pred[j]
            cnt = cnt+1
print(cnt)
result1 = []
for i in range(len(result)):
    result1.append(filename_list[result[i]])
print(result1[0:10])
df = pd.DataFrame({'file_name':sub['file_name'],'label':result1})
now = time.strftime("%Y%m%d_%H%M%S",time.localtime(time.time())) 
fname="D:\\下载\\submit_"+now+r".csv"    
df.to_csv(fname, index=False)


6835
['yes', 'on', 'cat', 'stop', 'no', 'off', 'four', 'on', 'no', 'happy']
